In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('FINAL.csv')

In [4]:
df['isOne'] = df['runs_off_bat'].apply(lambda x: 1 if x == 1 else 0)
df['isTwo'] = df['runs_off_bat'].apply(lambda x: 1 if x == 2 else 0)
df['isThree'] = df['runs_off_bat'].apply(lambda x: 1 if x == 3 else 0)
df['isFour'] = df['runs_off_bat'].apply(lambda x: 1 if x == 4 else 0)
df['isSix'] = df['runs_off_bat'].apply(lambda x: 1 if x == 6 else 0)
df['Boundary'] = df['runs_off_bat'].apply(lambda x: 1 if x == 6 or x==4 else 0)
    

In [5]:
df.columns

Index(['match_id', 'innings', 'Unnamed: 2', 'season', 'start_date', 'venue',
       'ball', 'batting_team', 'bowling_team', 'striker', 'non_striker',
       'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs', 'byes',
       'legbyes', 'penalty', 'wicket_type', 'player_dismissed',
       'other_wicket_type', 'other_player_dismissed', 'bowler_type',
       'Total_runs', 'Extraballs', 'over_type', 'number_of_wicket',
       'number_of_dotballs', 'Batting_type', 'isOne', 'isTwo', 'isThree',
       'isFour', 'isSix', 'Boundary'],
      dtype='object')

In [6]:
def isOut(player_dismissed):
    if player_dismissed == '0':
        return 0
    else:
        return 1

In [7]:
df['wicket'] = df.apply(lambda x: isOut(x['player_dismissed']), axis = 1)

In [8]:
data=df.groupby(['striker','over_type']).agg({'runs_off_bat':'sum','match_id':'nunique','wicket':'sum','Boundary':'sum'}).reset_index().rename({'runs_off_bat':'Runs_scored','match_id':'Match','wicket':'dismissed'},axis=1)
balls_faced=df.groupby(['striker','over_type']).agg({'match_id':'count'}).reset_index().rename({'match_id':'Balls_faced'},axis=1)
final_data=pd.merge(data,balls_faced,on=['striker','over_type'])

In [9]:
final_data

,striker,over_type,Runs_scored,Match,dismissed,Boundary,Balls_faced
0,A Ashish Reddy,Death overs,192,19,11,21,129
1,A Ashish Reddy,Middle overs,83,8,3,9,60
2,A Ashish Reddy,PowerPlay,5,1,1,1,7
3,A Chandila,Death overs,4,2,1,0,7
4,A Chopra,Middle overs,26,3,3,3,34
...,...,...,...,...,...,...,...
1214,Yuvraj Singh,Middle overs,1960,109,64,247,1646
1215,Yuvraj Singh,PowerPlay,193,33,12,31,197
1216,Z Khan,Death overs,101,23,11,12,98
1217,Z Khan,Middle overs,16,7,3,1,40


In [10]:
final_data['Average']=final_data['Runs_scored']/final_data['dismissed']
final_data['Runs_per_innings']=final_data['Runs_scored']/final_data['Match']
final_data['Striker_rate']=(final_data['Runs_scored']/final_data['Balls_faced'])*100
final_data['Balls_per_boundary']=final_data['Balls_faced']/final_data['Boundary']
final_data.drop(['Runs_scored','Match','dismissed','Boundary','Balls_faced'],axis=1,inplace=True)

In [11]:
final_data.replace(np.inf, 0, inplace=True)

In [12]:
final_data

,striker,over_type,Average,Runs_per_innings,Striker_rate,Balls_per_boundary
0,A Ashish Reddy,Death overs,17.454545,10.105263,148.837209,6.142857
1,A Ashish Reddy,Middle overs,27.666667,10.375000,138.333333,6.666667
2,A Ashish Reddy,PowerPlay,5.000000,5.000000,71.428571,7.000000
3,A Chandila,Death overs,4.000000,2.000000,57.142857,0.000000
4,A Chopra,Middle overs,8.666667,8.666667,76.470588,11.333333
...,...,...,...,...,...,...
1214,Yuvraj Singh,Middle overs,30.625000,17.981651,119.076549,6.663968
1215,Yuvraj Singh,PowerPlay,16.083333,5.848485,97.969543,6.354839
1216,Z Khan,Death overs,9.181818,4.391304,103.061224,8.166667
1217,Z Khan,Middle overs,5.333333,2.285714,40.000000,40.000000


In [22]:
data=pd.pivot_table(final_data,index=['striker'],columns=['over_type']).reset_index()

In [23]:
data.head()

striker     Average                         \
over_type                 Death overs Middle overs PowerPlay   
0          A Ashish Reddy   17.454545    27.666667       5.0   
1              A Chandila    4.000000          NaN       NaN   
2                A Chopra         NaN     8.666667      13.5   
3             A Choudhary   12.000000     0.000000       NaN   
4             A Dananjaya    0.000000          NaN       NaN   

          Balls_per_boundary                        Runs_per_innings  \
over_type        Death overs Middle overs PowerPlay      Death overs   
0                   6.142857     6.666667      7.00        10.105263   
1                   0.000000          NaN       NaN         2.000000   
2                        NaN    11.333333     10.25              NaN   
3                   9.500000     0.000000       NaN         8.000000   
4                   0.000000          NaN       NaN         4.000000   

                                 Striker_rate                          
over_type Middle overs PowerPlay  Death overs Middle overs  PowerPlay  
0            10.375000       5.0   148.837209   138.333333  71.428571  
1                  NaN       NaN    57.142857          NaN        NaN  
2             8.666667       5.4          NaN    76.470588  65.853659  
3             1.000000       NaN   126.315789   100.000000        NaN  
4                  NaN       NaN    80.000000          NaN        NaN

In [25]:
data.columns = [' '.join(col).strip() for col in data.columns.values]

In [27]:
data.head()

,striker,Average Death overs,Average Middle overs,Average PowerPlay,Balls_per_boundary Death overs,Balls_per_boundary Middle overs,Balls_per_boundary PowerPlay,Runs_per_innings Death overs,Runs_per_innings Middle overs,Runs_per_innings PowerPlay,Striker_rate Death overs,Striker_rate Middle overs,Striker_rate PowerPlay
0,A Ashish Reddy,17.454545,27.666667,5.0,6.142857,6.666667,7.00,10.105263,10.375000,5.0,148.837209,138.333333,71.428571
1,A Chandila,4.000000,NaN,NaN,0.000000,NaN,NaN,2.000000,NaN,NaN,57.142857,NaN,NaN
2,A Chopra,NaN,8.666667,13.5,NaN,11.333333,10.25,NaN,8.666667,5.4,NaN,76.470588,65.853659
3,A Choudhary,12.000000,0.000000,NaN,9.500000,0.000000,NaN,8.000000,1.000000,NaN,126.315789,100.000000,NaN
4,A Dananjaya,0.000000,NaN,NaN,0.000000,NaN,NaN,4.000000,NaN,NaN,80.000000,NaN,NaN


In [28]:
data.replace(np.NaN, 0, inplace=True)

In [29]:
data.head()

,striker,Average Death overs,Average Middle overs,Average PowerPlay,Balls_per_boundary Death overs,Balls_per_boundary Middle overs,Balls_per_boundary PowerPlay,Runs_per_innings Death overs,Runs_per_innings Middle overs,Runs_per_innings PowerPlay,Striker_rate Death overs,Striker_rate Middle overs,Striker_rate PowerPlay
0,A Ashish Reddy,17.454545,27.666667,5.0,6.142857,6.666667,7.00,10.105263,10.375000,5.0,148.837209,138.333333,71.428571
1,A Chandila,4.000000,0.000000,0.0,0.000000,0.000000,0.00,2.000000,0.000000,0.0,57.142857,0.000000,0.000000
2,A Chopra,0.000000,8.666667,13.5,0.000000,11.333333,10.25,0.000000,8.666667,5.4,0.000000,76.470588,65.853659
3,A Choudhary,12.000000,0.000000,0.0,9.500000,0.000000,0.00,8.000000,1.000000,0.0,126.315789,100.000000,0.000000
4,A Dananjaya,0.000000,0.000000,0.0,0.000000,0.000000,0.00,4.000000,0.000000,0.0,80.000000,0.000000,0.000000


In [38]:
max_avg_deathovers=max(data['Average Death overs'])
min_avg_deathovers=min(data['Average Death overs'])

max_avg_middleovers=max(data['Average Middle overs'])
min_avg_middleovers=min(data['Average Middle overs'])

max_avg_powerplay=max(data['Average PowerPlay'])
min_avg_powerplay=min(data['Average PowerPlay'])

max_bpb_deathovers=max(data['Balls_per_boundary Death overs'])
min_bpb_deathovers=min(data['Balls_per_boundary Death overs'])

max_bpb_middleovers=max(data['Balls_per_boundary Middle overs'])
min_bpb_middleovers=min(data['Balls_per_boundary Middle overs'])

max_bpb_powerplay=max(data['Balls_per_boundary PowerPlay'])
min_bpb_powerplay=min(data['Balls_per_boundary PowerPlay'])

max_rpi_deathovers=max(data['Runs_per_innings Death overs'])
min_rpi_deathovers=min(data['Runs_per_innings Death overs'])

max_rpi_middleovers=max(data['Runs_per_innings Middle overs'])
min_rpi_middleovers=min(data['Runs_per_innings Middle overs'])

max_rpi_powerplay=max(data['Runs_per_innings PowerPlay'])
min_rpi_powerplay=min(data['Runs_per_innings PowerPlay'])

max_sr_deathovers=max(data['Striker_rate Death overs'])
min_sr_deathovers=min(data['Striker_rate Death overs'])

max_sr_middleovers=max(data['Striker_rate PowerPlay'])
min_sr_middleovers=min(data['Striker_rate PowerPlay'])

max_sr_powerplay=max(data['Striker_rate PowerPlay'])
min_sr_powerplay=min(data['Striker_rate PowerPlay'])

In [59]:
data.round(2)

,striker,Average Death overs,Average Middle overs,Average PowerPlay,Balls_per_boundary Death overs,Balls_per_boundary Middle overs,Balls_per_boundary PowerPlay,Runs_per_innings Death overs,Runs_per_innings Middle overs,Runs_per_innings PowerPlay,Striker_rate Death overs,Striker_rate Middle overs,Striker_rate PowerPlay
0,A Ashish Reddy,17.45,27.67,5.00,6.14,6.67,7.00,10.11,10.38,5.00,148.84,138.33,71.43
1,A Chandila,4.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,57.14,0.00,0.00
2,A Chopra,0.00,8.67,13.50,0.00,11.33,10.25,0.00,8.67,5.40,0.00,76.47,65.85
3,A Choudhary,12.00,0.00,0.00,9.50,0.00,0.00,8.00,1.00,0.00,126.32,100.00,0.00
4,A Dananjaya,0.00,0.00,0.00,0.00,0.00,0.00,4.00,0.00,0.00,80.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,YV Takawale,0.00,14.00,24.20,2.60,8.25,6.85,14.50,14.00,15.12,223.08,127.27,88.32
543,Yashpal Singh,11.00,9.50,17.00,5.00,20.00,18.50,11.00,6.33,8.50,110.00,95.00,45.95
544,Younis Khan,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,42.86
545,Yuvraj Singh,18.21,30.62,16.08,4.09,6.66,6.35,12.79,17.98,5.85,165.11,119.08,97.97


In [68]:
data['avg_deathovers']=data['Average Death overs'].apply(lambda x:(x-min_avg_deathovers)/(max_avg_deathovers-min_avg_deathovers))
data['avg_middleovers']=data['Average Middle overs'].apply(lambda x:(x-min_avg_middleovers)/(max_avg_middleovers-min_avg_middleovers))
data['avg_powerplay']=data['Average PowerPlay'].apply(lambda x:(x-min_avg_powerplay)/(max_avg_powerplay-min_avg_powerplay))

data['bpb_deathovers']=data['Balls_per_boundary Death overs'].apply(lambda x:(x-min_bpb_deathovers)/(max_bpb_deathovers-min_bpb_deathovers))
data['bpb_middleovers']=data['Balls_per_boundary Middle overs'].apply(lambda x:(x-min_bpb_middleovers)/(max_bpb_middleovers-min_bpb_middleovers))
data['bpb_powerplay']=data['Balls_per_boundary PowerPlay'].apply(lambda x:(x-min_bpb_powerplay)/(max_bpb_powerplay-min_bpb_powerplay))

data['rpi_deathovers']=data['Runs_per_innings Death overs'].apply(lambda x:(x-min_rpi_deathovers)/(max_rpi_deathovers-min_rpi_deathovers))
data['rpi_middleovers']=data['Runs_per_innings Middle overs'].apply(lambda x:(x-min_rpi_middleovers)/(max_rpi_middleovers-min_rpi_middleovers))
data['rpi_powerplay']=data['Runs_per_innings PowerPlay'].apply(lambda x:(x-min_rpi_powerplay)/(max_rpi_powerplay-min_rpi_powerplay))

data['sr_deathovers']=data['Striker_rate Death overs'].apply(lambda x:(x-min_sr_deathovers)/(max_sr_deathovers-min_sr_deathovers))
data['sr_middleovers']=data['Striker_rate Middle overs'].apply(lambda x:(x-min_sr_middleovers)/(max_sr_middleovers-min_sr_middleovers))
data['sr_powerplay']=data['Striker_rate PowerPlay'].apply(lambda x:(x-min_sr_powerplay)/(max_sr_powerplay-min_sr_powerplay))


In [104]:
batsman_name=input('Enter the name: ')
player_avg_deathovers=data[data.striker==batsman_name].avg_deathovers.item()
player_avg_middleovers=data[data.striker==batsman_name].avg_middleovers.item()
player_avg_powerplay=data[data.striker==batsman_name].avg_powerplay.item()

player_bpb_deathovers=data[data.striker==batsman_name].bpb_deathovers.item()
player_bpb_middleovers=data[data.striker==batsman_name].bpb_middleovers.item()
player_bpb_powerplay=data[data.striker==batsman_name].bpb_powerplay.item()

player_rpi_deathovers=data[data.striker==batsman_name].rpi_deathovers.item()
player_rpi_middleovers=data[data.striker==batsman_name].rpi_middleovers.item()
player_rpi_powerplay=data[data.striker==batsman_name].rpi_powerplay.item()

player_sr_deathovers=data[data.striker==batsman_name].sr_deathovers.item()
player_sr_middleovers=data[data.striker==batsman_name].sr_middleovers.item()
player_sr_powerplay=data[data.striker==batsman_name].sr_powerplay.item()

avg_death1=np.array(data.avg_deathovers)
avg_death2= np.empty(len(avg_death1))
avg_death2.fill(player_avg_deathovers)

avg_middle1=np.array(data.avg_middleovers)
avg_middle2= np.empty(len(avg_middle1))
avg_middle2.fill(player_avg_middleovers)

avg_power1=np.array(data.avg_powerplay)
avg_power2= np.empty(len(avg_power1))
avg_power2.fill(player_avg_powerplay)

bpb_death1=np.array(data.bpb_deathovers)
bpb_death2= np.empty(len(bpb_death1))
bpb_death2.fill(player_bpb_deathovers)

bpb_middle1=np.array(data.bpb_middleovers)
bpb_middle2= np.empty(len(bpb_middle1))
bpb_middle2.fill(player_bpb_middleovers)

bpb_power1=np.array(data.bpb_powerplay)
bpb_power2= np.empty(len(bpb_power1))
bpb_power2.fill(player_bpb_powerplay)


rpi_death1=np.array(data.rpi_deathovers)
rpi_death2= np.empty(len(rpi_death1))
rpi_death2.fill(player_rpi_deathovers)

rpi_middle1=np.array(data.rpi_middleovers)
rpi_middle2= np.empty(len(rpi_middle1))
rpi_middle2.fill(player_rpi_middleovers)

rpi_power1=np.array(data.rpi_powerplay)
rpi_power2= np.empty(len(rpi_power1))
rpi_power2.fill(player_rpi_powerplay)

sr_death1=np.array(data.sr_deathovers)
sr_death2= np.empty(len(sr_death1))
sr_death2.fill(player_sr_deathovers)

sr_middle1=np.array(data.sr_middleovers)
sr_middle2= np.empty(len(sr_middle1))
sr_middle2.fill(player_sr_middleovers)

sr_power1=np.array(data.sr_powerplay)
sr_power2= np.empty(len(sr_power1))
sr_power2.fill(player_sr_powerplay)

avg_death3=abs(avg_death1-avg_death2)**2
avg_middle3=abs(avg_middle1-avg_middle2)**2
avg_power3=abs(avg_power1-avg_power2)**2

bpb_death3=abs(bpb_death1-bpb_death2)**2
bpb_middle3=abs(bpb_middle1-bpb_middle2)**2
bpb_power3=abs(bpb_power1-bpb_power2)**2

rpi_death3=abs(rpi_death1-rpi_death2)**2
rpi_middle3=abs(rpi_middle1-rpi_middle2)**2
rpi_power3=abs(rpi_power1-rpi_power2)**2

sr_death3=abs(sr_death1-sr_death2)**2
sr_middle3=abs(sr_middle1-sr_middle2)**2
sr_power3=abs(sr_power1-sr_power2)**2
distance=avg_death3+avg_middle3+avg_power3+bpb_death3+bpb_middle3+bpb_power3+rpi_death3+rpi_middle3+rpi_power3+sr_death3+sr_middle3+sr_power3
dist=np.sqrt(distance)
data['distance']=dist
final_data=data.drop(['Average Death overs', 'Average Middle overs',
       'Average PowerPlay', 'Balls_per_boundary Death overs',
       'Balls_per_boundary Middle overs', 'Balls_per_boundary PowerPlay',
       'Runs_per_innings Death overs', 'Runs_per_innings Middle overs',
       'Runs_per_innings PowerPlay', 'Striker_rate Death overs',
       'Striker_rate Middle overs', 'Striker_rate PowerPlay'],axis=1)
min_d = min(final_data.distance)
max_d = max(final_data.distance)

final_data['similarity'] = final_data['distance'].apply(lambda x: 100*(1 - (x-min_d)/(max_d-min_d)))
final_data.sort_values('similarity', ascending= False).head(10)


Enter the name: S Dhawan


,striker,avg_deathovers,avg_middleovers,avg_powerplay,bpb_deathovers,bpb_middleovers,bpb_powerplay,rpi_deathovers,rpi_middleovers,rpi_powerplay,sr_deathovers,sr_middleovers,sr_powerplay,distance,similarity
416,S Dhawan,0.305303,0.397846,0.465393,0.126614,0.114695,0.208843,0.335833,0.666204,0.525189,0.526797,0.357292,0.340915,0.000000,100.000000
42,AM Rahane,0.213939,0.391597,0.402609,0.134503,0.145802,0.225201,0.326852,0.631192,0.494192,0.511594,0.341392,0.321531,0.129433,91.366113
447,SK Raina,0.268182,0.395438,0.441043,0.126765,0.110643,0.183761,0.345213,0.650217,0.389975,0.547679,0.369801,0.369245,0.150479,89.962193
300,ML Hayden,0.300000,0.433333,0.545736,0.094737,0.095878,0.183549,0.330000,0.684211,0.623958,0.611111,0.393413,0.355068,0.168760,88.742759
289,MEK Hussey,0.248485,0.429194,0.555854,0.096491,0.118126,0.251430,0.315385,0.742836,0.501818,0.621212,0.365967,0.299079,0.180841,87.936902
464,SR Tendulkar,0.200000,0.319457,0.560243,0.101504,0.125779,0.214969,0.235714,0.598982,0.494372,0.543210,0.326047,0.315557,0.210718,85.943932
514,V Kohli,0.244697,0.471306,0.458183,0.097349,0.128693,0.234803,0.417672,0.695091,0.424074,0.656504,0.353245,0.321122,0.214094,85.718741
407,RV Uthappa,0.169697,0.316444,0.393282,0.119433,0.107318,0.210175,0.306250,0.570636,0.450122,0.553672,0.359153,0.343005,0.216094,85.585289
465,SR Watson,0.157754,0.374394,0.442073,0.107237,0.077001,0.219877,0.245833,0.649460,0.464379,0.603272,0.430261,0.314241,0.220783,85.272499
467,SS Iyer,0.225253,0.349465,0.409378,0.105263,0.117560,0.248077,0.371667,0.662779,0.362366,0.599462,0.360801,0.298560,0.220870,85.266704
